In [1]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [2]:
import numpy as np
import json
from equistore import Labels, TensorBlock, TensorMap
from utils.builder import TensorBuilder
import ase.io
from itertools import product
from utils.acdc_mini import acdc_standardize_keys, cg_increment, cg_combine, _remove_suffix
from utils.clebsh_gordan import ClebschGordanReal
from utils.hamiltonians import hamiltonian_features
import matplotlib.pyplot as plt

from utils.librascal import  RascalSphericalExpansion, RascalPairExpansion
from rascal.representations import SphericalExpansion
import copy
from utils.model_hamiltonian import *
from itertools import product

In [3]:
from utils.mp_utils import * 

In [4]:
frames1 = ase.io.read("./data/hamiltonian/water-hamiltonian/water_coords_1000.xyz",":2")
# frames2 = ase.io.read("./data/hamiltonian/ethanol-hamiltonian/ethanol_4500.xyz",":1")
# frames3= [ase.build.molecule('NH3')]
# frames = frames1+frames2#+frames3#+frames2
# frames = ase.io.read('../feats_MP/methane/data/methane_40k.xyz',':10')
frames = frames1
for f in frames:
    f.cell = [100,100,100]
    f.positions += 50

In [5]:
rascal_hypers = {
    "interaction_cutoff": 3.5,
    "cutoff_smooth_width": 0.3,
    "max_radial": 2,
    "max_angular": 2,
    "gaussian_sigma_type": "Constant",
    "compute_gradients":  False,
#     "expansion_by_species_method": "user defined",
#     "global_species": [1,6,8,7]
    
}

spex = RascalSphericalExpansion(rascal_hypers)
rhoi = spex.compute(frames)

In [6]:
lmax = rascal_hypers["max_angular"]+2
cg = ClebschGordanReal(lmax)

In [7]:
pairs = RascalPairExpansion(rascal_hypers)
gij = pairs.compute(frames)

In [8]:
rho1i = acdc_standardize_keys(rhoi)
rho1i.keys_to_properties(['species_neighbor'])
gij =  acdc_standardize_keys(gij)

In [9]:
rho2i =  cg_combine(rho1i, rho1i, clebsch_gordan=cg, lcut=lmax, other_keys_match = ['species_center'])

In [10]:
rhoii1i2_nu0 = cg_combine(gij, gij, clebsch_gordan=cg,lcut=lmax, other_keys_match=['species_center'])

In [11]:
rhoii1i2_nu0

TensorMap with 64 blocks
keys: ['order_nu' 'inversion_sigma' 'spherical_harmonics_l' 'species_center' 'species_neighbor_a' 'species_neighbor_b']
           2             1                    0                   1                 1                   1
           2             1                    0                   1                 1                   8
           2             1                    1                   1                 1                   1
        ...
           2             1                    4                   8                 8                   1
           2            -1                    3                   8                 8                   8
           2             1                    4                   8                 8                   8

In [15]:
rhoii1i2_nu1 =  cg_combine(rho1i, rhoii1i2_nu0, clebsch_gordan=cg, lcut=lmax, other_keys_match = ['species_center'])

In [16]:
rhoii1i2_nu1

TensorMap with 80 blocks
keys: ['order_nu' 'inversion_sigma' 'spherical_harmonics_l' 'species_center' 'species_neighbor_a' 'species_neighbor_b']
           3             1                    0                   1                 1                   1
           3             1                    0                   1                 1                   8
           3             1                    1                   1                 1                   1
        ...
           3            -1                    4                   8                 1                   8
           3            -1                    4                   8                 8                   1
           3            -1                    4                   8                 8                   8

In [17]:
## going from three center to rho_i append to the property dimension species 
def contract_three_center(rhoii1i2):
    if len(rhoii1i2.keys.dtype)>4:
        rhoii1i2.keys_to_properties('species_neighbor_b')
        rhoii1i2.keys_to_properties('species_neighbor_a')
    contracted_rhoii1i2_blocks=[]

    property_names = rhoii1i2.property_names
    for key,block in rhoii1i2:
        contract_values=[]
        contract_properties=[]
        contract_samples=list(product(np.unique(block.samples['structure']), np.unique(block.samples['center'])))
        for isample, sample in enumerate(contract_samples):
            sample_idx = [idx for idx, tup in enumerate(block.samples) if tup[0]==sample[0] and tup[1]==sample[1]]
            contract_values.append(block.values[sample_idx].sum(axis=0)) # sum i_1, i_2
#             print(key, block.values[sample_idx].sum(axis=0).shape,sample,sample_idx)

        new_block = TensorBlock(values = np.asarray(contract_values),#.reshape(len(contract_samples),contract_values[0].shape[1] ,-1),
                                        samples = Labels(['structure', 'center'], np.asarray(contract_samples, np.int32)), 
                                         components = block.components,
                                         properties= Labels(list(property_names), block.properties.asarray())
                                )

        contracted_rhoii1i2_blocks.append(new_block)

    contracted_rhoii1i2 = TensorMap(rhoii1i2.keys, contracted_rhoii1i2_blocks)
    return contracted_rhoii1i2

In [18]:
contracted_rhoii1i2 = contract_three_center(rhoii1i2_nu0)

### sanity check - 
we need to compare the contraction with rho2, but the property labels of the two tensor maps are in diff orders
here's a naive solution to find the index of prop label in one tensor in another

In [19]:
def compare_with_rho2i(contracted_rhoii1i2, rho2i):
    assert len(rho2i) == len(contracted_rhoii1i2)
    for rho2_k, rho2_b in rho2i:
        contracted_block = contracted_rhoii1i2.block(order_nu = rho2_k[0], inversion_sigma = rho2_k[1], spherical_harmonics_l = rho2_k[2], species_center = rho2_k[3])
        idx = []
        cidx = []
        for i,p in enumerate(rho2_b.properties):
            find_p = (p['species_neighbor_a'], p['species_neighbor_b'], p['n_1_a'], p['k_2'], p['n_1_b'], p['l_2'])
        #     print(p, find_p)
            for ip,cp in enumerate(contracted_block.properties):
                if tuple(find_p) == tuple(cp) :
                    idx.append(i)
                    cidx.append(ip)
                    break
        print(rho2_k, np.linalg.norm(rho2_b.values[:,:,idx] - contracted_block.values[:,:,cidx]))

compare_with_rho2i(contracted_rhoii1i2, rho2i)

(2, 1, 0, 1) 2.3412456464305706e-20
(2, 1, 1, 1) 1.8707075487275463e-20
(2, 1, 2, 1) 1.9540399352564176e-20
(2, 1, 0, 8) 2.938484138683137e-20
(2, 1, 1, 8) 1.3797231381027056e-20
(2, 1, 2, 8) 2.6540990298228056e-20
(2, -1, 1, 1) 0.0
(2, -1, 2, 1) 0.0
(2, 1, 3, 1) 0.0
(2, -1, 1, 8) 4.290085066403457e-21
(2, -1, 2, 8) 1.1003280752469429e-20
(2, 1, 3, 8) 8.852469447498877e-21
(2, -1, 3, 1) 0.0
(2, 1, 4, 1) 0.0
(2, -1, 3, 8) 7.727960980828157e-21
(2, 1, 4, 8) 8.668935063650161e-21


so just summing over i1, i2 of the three center feature yields power spectrum as it should!

## old